In [1]:
!pip install datasets evaluate transformers seqeval pandas accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=2bb715f77ace8523b6cf19b8edabbe1548314e22ac5de3bdc130c06e925d535c
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import evaluate
import datasets
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification
import torch

COLAB = 'google.colab' in sys.modules
if COLAB:
    torch.cuda.is_available
COLAB

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512,garbage_collection_threshold:0.9"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
if not COLAB:
    from IPython.core.display import display, HTML
    display(HTML("<style>.container { width:80% !important; }</style>"))
    %load_ext autoreload
    %autoreload 2

    sys.path.append("../../")
else:
    if not os.path.exists("ner"):
        from google.colab import drive
        drive.mount("/content/drive")

        !cp -r /content/drive/MyDrive/colab_folder/notebooks .

         # with current datasets package (2.4) create_train_test_split_hf_dataset does not work

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
if COLAB:
    %load_ext tensorboard

In [6]:

tru_label = ['O',
              'B-cond',
              'I-cond',
              'B-des',
              'I-des',
              'B-subj',
              'I-subj',
              "B-group_A",
              "I-group_A",
              "B-group_B",
              "I-group_B",
              "B-group_C",
              "I-group_C",
              'B-group_D',
              'I-group_D']

id2label = {i:label for i, label in enumerate(tru_label) }
label2id = {label:i for i, label in enumerate(tru_label) }

#### Reading the datasest

In [7]:

DATA_PATH = Path("/content/drive/MyDrive/colab_folder/datasets")
MODELS_BASE_PATH = Path("/content/drive/MyDrive/colab_folder/models")


In [8]:
import json
dataset_date = "2023-11-20"

def read_file(DATA_PATH, doc):
    file_path = doc + '.json'
    file_name = os.path.join(DATA_PATH, file_path)

    with open(file_name, "r", encoding="utf-8") as f:
        return json.load(f)

In [9]:
file_names = ["final_labelled_abstract_22_11_2023_2.0","final_labelled_abstract_method_22_11_2023_2.0", "final_labelled_abstract_method_50_22_11_2023_2.0", "final_labelled_abstract_method_500_22_11_2023_2.0" ]
abstract= read_file(DATA_PATH, file_names[0])
abstract_methods = read_file(DATA_PATH, file_names[1])
splitted_500_abstract_methods_context = read_file(DATA_PATH, file_names[3])
splitted_50_records_method_context =read_file(DATA_PATH, file_names[2])




In [10]:
json.dumps(abstract_methods[0])

'{"pmid": "16960863", "ner_tags": [0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 7, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 9, 0, 0, 7, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 9, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
json.dumps(splitted_50_records_method_context[12])


'{"pmid": "11145488", "ner_tags": [0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 5, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 9, 0, 0, 0, 0, 0, 0, 7, 0, 9, 0,

#### FUNCTIONS

In [12]:
label_dict = {'O': 0, 'B-condition': 1, 'I-condition': 2,
              'B-design': 3, 'I-design': 4,
              'B-subjects': 5,'I-subjects': 6,
              "B-group_A": 7, "I-group_A": 8,
              "B-group_B": 9, "I-group_B": 10,
              "B-group_C": 11,"I-group_C": 12,
              'B-group_D': 13,'I-group_D': 14
              }
def tokenize_align_labels(example, label_all_tokens = True):
    # tokenized_input = tokenizer(example['tokens'], truncation = True, is_split_into_words=True, max_length = 512)
    tokenized_input = tokenizer(example['tokens'], truncation = True, is_split_into_words=True)

    labels = []
    for i , label in enumerate(example['ner_tags']):
        word_ids = tokenized_input.word_ids(batch_index=i)
        previous_word_idx = None

        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_ids!= previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_input['labels'] = labels
    return tokenized_input

from collections import Counter

def get_key(val):
    for key, value in label_dict.items():
        if val == value:
            return key
    return "key doesn't exist"




def count_values(lst):
    # Use Counter to count occurrences of each value in the list
    value_counts = Counter(lst)

    # Convert the Counter to a dictionary for a more readable output
    count_dict = dict(value_counts)

    return count_dict

def explore_annotations(dataset):
    all_annotations = []

    for records in dataset:
        temp_tag = records['ner_tags']
        all_annotations.extend(temp_tag)
    print(f'There are {len(all_annotations)} annotations in the dataset')

    return [get_key(x) for x in all_annotations]

def normalize_tag_name(tag):
    if tag in [ 'condition', 'condition_1', 'condition_2', 'condition_3', 'condition_4', 'condition_5', 'condition_6', 'condition_7']:
        tag= 'condition'
    elif tag in ['design', 'design_1', 'design_2', 'design_3', 'design_4', 'design_5']:
        tag = 'design'
    elif tag in ['subject_1', 'subject_2', 'subjects', 'subjects_1', 'subjects_2', 'subjects_3', 'subjects_4', 'subjects_5', 'subjekts', 'sujects']:
        tag = 'subjects'
    elif tag in [ 'group_A', 'group_A_1', 'group_A_2', 'group_A_3', 'group_A_4']:
        tag = 'group_A'
    elif tag in ['groub_B_1', 'group_B', 'group_B _2', 'group_B_1', 'group_B_2', 'group_B_3', 'group_B_4' ]:
        tag = 'group_B'
    elif tag in [ 'group_C', 'group_C_1', 'group_C_2', 'group_C_3' ]:
        tag = 'group_C'
    elif tag in [ 'group_D', 'group_D_1', 'group_D_2', 'group_D_3']:
        tag = 'group_D'
    return tag
def create_datasets(doc_dict, labels):
    temp = Dataset.from_pandas(pd.DataFrame(doc_dict))
    temp = temp.cast_column("ner_tags", datasets.Sequence(datasets.ClassLabel(names=labels)))
    return temp

def train_test_split(data_set, train_test_size, validation_size):
    from datasets import DatasetDict
    train_testvalid = data_set.train_test_split(test_size=train_test_size)

    # Split the test to half test, half valid
    test_valid = train_testvalid['test'].train_test_split(test_size=validation_size)

    #Gather everything into one dataset dictionary
    train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

    return train_test_valid_dataset



#### hyperparameters

In [13]:
model_checkpoint = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length = 512)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, ignore_mismatched_sizes=True, id2label = id2label, label2id = label2id)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = evaluate.load('seqeval')

In [15]:
from typing import Tuple
from sklearn.metrics import precision_score, recall_score
from functools import partial

def token_f1_cond(true, pred, labels):
    class_scores = zip(labels, precision_score(true, pred, labels=labels, average=None, zero_division=True),
                       recall_score(true, pred, labels=labels, average=None))
    result = {label: {"f1": get_f1(prec, rec), "p": prec, "r": rec} for label, prec, rec in class_scores}
    return result
def get_f1(prec, rec):
    return 2 * prec * rec / (prec + rec)

def compute_metrics_by_token_swt(p: Tuple[list, list], label_list):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [label_list[p] for prediction, label in zip(predictions, labels) for (p, l) in
                        zip(prediction, label) if l != -100]
    true_labels = [label_list[l] for prediction, label in zip(predictions, labels) for (p, l) in zip(prediction, label)
                   if l != -100]

    metrics_report = token_f1_cond(true_labels, true_predictions, label_list)
    metrics_report_f1 = {"f1_" + k: v["f1"] for k, v in metrics_report.items() if k != "O"}
    return metrics_report_f1
compute_metrics_by_token_swt = partial(compute_metrics_by_token_swt, label_list=tru_label)


In [16]:
from transformers import TrainingArguments, Trainer

model_max_length = 512

if COLAB:
    batch_size = 16 # for p-100 16 is ok. For T4: 12
    if model_max_length <= 384:
      # at least 15 GB gpu
        batch_size = 16
    gradient_accumulation_steps = 1
    num_train_epochs = 8
else:
    batch_size = 4
    if model_max_length==256:
        batch_size = 9

    gradient_accumulation_steps = 4
    num_train_epochs = 5

learning_rate = 5e-5
weight_decay = 0.003

print(f"bs: {batch_size}, model_max_length: {model_max_length}, gradient_acc_steps: {gradient_accumulation_steps}, \
n_epochs: {num_train_epochs}, lr: {learning_rate}")

bs: 16, model_max_length: 512, gradient_acc_steps: 1, n_epochs: 8, lr: 5e-05


### ABSTRACT ONLY

In [17]:
for token, label in zip(abstract[0]['tokens'], abstract[0]['ner_tags']):
    print(f"{token:_<40} {label}")

effects_________________________________ 0
of______________________________________ 0
rivastigmine____________________________ 7
in______________________________________ 0
patients________________________________ 0
with____________________________________ 0
and_____________________________________ 0
without_________________________________ 0
visual__________________________________ 0
hallucinations__________________________ 0
in______________________________________ 0
dementia________________________________ 0
associated______________________________ 0
with____________________________________ 0
parkinson’s_____________________________ 0
disease_________________________________ 0
._______________________________________ 0
we______________________________________ 0
aimed___________________________________ 0
to______________________________________ 0
determine_______________________________ 0
prospectively___________________________ 0
whether_________________________________ 0
rivastigmin

In [18]:
for token, label in zip(abstract[10]['tokens'], abstract[10]['ner_tags']):
    print(f"{token:_<40} {label}")

five-year_______________________________ 0
follow-up_______________________________ 0
of______________________________________ 0
early___________________________________ 0
lisuride________________________________ 0
and_____________________________________ 0
levodopa________________________________ 0
combination_____________________________ 0
therapy_________________________________ 0
versus__________________________________ 0
levodopa________________________________ 0
monotherapy_____________________________ 0
in______________________________________ 0
de______________________________________ 0
novo____________________________________ 0
parkinson’s_____________________________ 0
disease_________________________________ 0
._______________________________________ 0
the_____________________________________ 0
french__________________________________ 0
lisuride________________________________ 0
study___________________________________ 0
group___________________________________ 0
.__________

##### count the annotations

In [19]:
tags = explore_annotations(abstract)
import pandas as pd
result = count_values(tags)
temp_df = pd.DataFrame(result.items(), columns=['label_id', 'count'])
temp_df.sort_values(by = ['count'], ascending=False)

There are 565899 annotations in the dataset


,label_id,count
0,O,533739
1,B-group_A,4820
10,I-condition,4648
7,B-group_B,3778
5,I-design,3711
2,B-subjects,3307
6,I-group_A,2739
9,B-condition,2044
11,I-group_B,2029
3,I-subjects,1839


In [20]:
len(abstract)

1549

#### Creating Huggingface Dataset

In [21]:

abstract_hf_ds = create_datasets(abstract, tru_label)

Casting the dataset:   0%|          | 0/1549 [00:00<?, ? examples/s]

In [22]:
abstract_hf_ds.features

{'pmid': Value(dtype='string', id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-cond', 'I-cond', 'B-des', 'I-des', 'B-subj', 'I-subj', 'B-group_A', 'I-group_A', 'B-group_B', 'I-group_B', 'B-group_C', 'I-group_C', 'B-group_D', 'I-group_D'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [23]:
abstract_hf_ds[0]


{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [24]:
abstract_hf_ds_train_test_valid = train_test_split(abstract_hf_ds, train_test_size=0.2, validation_size=0.5)


In [25]:
abstract_hf_ds_train_test_valid

DatasetDict({
    train: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 1239
    })
    test: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 155
    })
    valid: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 155
    })
})

In [26]:
tags = abstract_hf_ds_train_test_valid['train'].features['ner_tags'].feature.names
tags

['O',
 'B-cond',
 'I-cond',
 'B-des',
 'I-des',
 'B-subj',
 'I-subj',
 'B-group_A',
 'I-group_A',
 'B-group_B',
 'I-group_B',
 'B-group_C',
 'I-group_C',
 'B-group_D',
 'I-group_D']

#### Tokenizer Example and Testing

In [27]:
example_text = abstract_hf_ds_train_test_valid['train'][0]
example_text['tokens']

['safety',
 'and',
 'efficacy',
 'of',
 'high',
 'dose',
 'atg',
 'bolus',
 'administration',
 'on',
 'rewascularization',
 'in',
 'kidney',
 'graft',
 'patients‐‐long',
 'term',
 'results',
 '.',
 'background',
 ':',
 'various',
 'preparations',
 'of',
 'alg',
 '/',
 'atg',
 'have',
 'been',
 'used',
 'in',
 'clinical',
 'transplantation',
 'for',
 'more',
 'than',
 '30',
 'years',
 '.',
 'in',
 'recent',
 'years',
 'the',
 'number',
 'of',
 'high',
 'immunological',
 'risk',
 'patients',
 'has',
 'increased',
 'and',
 'biological',
 'agents',
 'are',
 'being',
 'used',
 'as',
 'induction',
 'therapy',
 '.',
 'the',
 'aim',
 'of',
 'this',
 'prospective',
 ',',
 'randomized',
 'study',
 'was',
 'to',
 'asses',
 'the',
 'safety',
 'and',
 'efficacy',
 'of',
 'a',
 'single',
 'high',
 'dose',
 'of',
 'antithymocyte',
 'globulin',
 '(',
 '9',
 'mg',
 '/',
 'kg',
 'atg',
 'fresenius',
 's',
 ')',
 'in',
 'cadaveric',
 'renal',
 'transplantation',
 '.',
 'the',
 'maintenance',
 'immunosupp

In [28]:

tokenized_input = tokenizer(example_text['tokens'], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

words_ids = tokenized_input.word_ids()
words_ids

[None,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 11,
 12,
 13,
 14,
 14,
 14,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 81,
 81,
 81,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 89,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 105,
 105,
 106,
 106,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,

In [29]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
tokens

['[CLS]',
 'safety',
 'and',
 'efficacy',
 'of',
 'high',
 'dose',
 'atg',
 'bolus',
 'administration',
 'on',
 'rew',
 '##ascular',
 '##ization',
 'in',
 'kidney',
 'graft',
 'patients',
 '‐',
 '‐',
 'long',
 'term',
 'results',
 '.',
 'background',
 ':',
 'various',
 'preparations',
 'of',
 'alg',
 '/',
 'atg',
 'have',
 'been',
 'used',
 'in',
 'clinical',
 'transplantation',
 'for',
 'more',
 'than',
 '30',
 'years',
 '.',
 'in',
 'recent',
 'years',
 'the',
 'number',
 'of',
 'high',
 'immunological',
 'risk',
 'patients',
 'has',
 'increased',
 'and',
 'biological',
 'agents',
 'are',
 'being',
 'used',
 'as',
 'induction',
 'therapy',
 '.',
 'the',
 'aim',
 'of',
 'this',
 'prospective',
 ',',
 'randomized',
 'study',
 'was',
 'to',
 'ass',
 '##es',
 'the',
 'safety',
 'and',
 'efficacy',
 'of',
 'a',
 'single',
 'high',
 'dose',
 'of',
 'antit',
 '##hy',
 '##mo',
 '##cyt',
 '##e',
 'globulin',
 '(',
 '9',
 'mg',
 '/',
 'kg',
 'atg',
 'fre',
 '##sen',
 '##ius',
 's',
 ')',
 'in'

In [30]:
len(example_text['ner_tags']), len(tokenized_input['input_ids'])

(340, 364)

#### testing tokenizer

In [31]:
q = tokenize_align_labels(abstract_hf_ds_train_test_valid['train'][0:1])
print(q)

{'input_ids': [[2, 5407, 1930, 4348, 1927, 2149, 3253, 9353, 14840, 4183, 1990, 17147, 3289, 2466, 1922, 5245, 7668, 2132, 226, 226, 2815, 2576, 2274, 18, 3872, 30, 3425, 9815, 1927, 4326, 19, 9353, 2162, 2252, 2251, 1922, 2535, 6530, 1958, 2253, 2254, 2627, 2739, 18, 1922, 3505, 2739, 1920, 2529, 1927, 2149, 11611, 2565, 2132, 2258, 2502, 1930, 3933, 4820, 2032, 3301, 2251, 1966, 4408, 3181, 18, 1920, 3774, 1927, 2052, 6315, 16, 5542, 2161, 1982, 1942, 2088, 1918, 1920, 5407, 1930, 4348, 1927, 43, 2957, 2149, 3253, 1927, 10310, 30338, 12492, 27932, 1021, 24366, 12, 29, 2786, 19, 3621, 9353, 2805, 17221, 11519, 61, 13, 1922, 28670, 4604, 6530, 18, 1920, 6291, 12910, 8607, 6994, 1927, 13039, 16, 5078, 23965, 2010, 25976, 4988, 1024, 12, 9076, 2256, 1920, 8776, 2935, 1942, 30499, 5773, 13, 16, 1930, 21002, 18, 3485, 19, 2860, 30, 2192, 12832, 6000, 1930, 11071, 5372, 16, 5637, 9336, 1985, 3063, 2460, 1920, 2161, 18, 2132, 1985, 5542, 1942, 9353, 12, 56, 33, 3040, 13, 2014, 1920, 2970, 23

In [32]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q['input_ids'][0]), q['labels'][0]):
    print(f'{token:_<40} {label}')


[CLS]___________________________________ -100
safety__________________________________ 0
and_____________________________________ 0
efficacy________________________________ 0
of______________________________________ 0
high____________________________________ 0
dose____________________________________ 0
atg_____________________________________ 7
bolus___________________________________ 8
administration__________________________ 0
on______________________________________ 0
rew_____________________________________ 0
##ascular_______________________________ 0
##ization_______________________________ 0
in______________________________________ 0
kidney__________________________________ 0
graft___________________________________ 0
patients________________________________ 0
‐_______________________________________ 0
‐_______________________________________ 0
long____________________________________ 0
term____________________________________ 0
results_________________________________ 0
._______

#### tokenizing dataset

In [33]:
tokenized_dataset = abstract_hf_ds_train_test_valid.map(tokenize_align_labels, batched = True, remove_columns=abstract_hf_ds_train_test_valid['train'].column_names)

Map:   0%|          | 0/1239 [00:00<?, ? examples/s]

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

#### Trainer

In [34]:
output_dir = r'/content/drive/MyDrive/colab_folder/models'
run_name = "pubmedbert_Abstract"
from transformers import Trainer, TrainingArguments
args = TrainingArguments(
    output_dir= output_dir,
    evaluation_strategy='epoch',
    eval_steps=1,
    save_strategy = 'epoch',
    learning_rate = learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    gradient_accumulation_steps=gradient_accumulation_steps,
    logging_strategy="epoch",
    save_total_limit=2,
    run_name =run_name,
    load_best_model_at_end=True
)


trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['valid'],
    data_collator = data_collator,
    tokenizer = tokenizer,


    compute_metrics = compute_metrics_by_token_swt

)

In [35]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1239
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 155
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 155
    })
})

In [36]:
trainer.train()

model.save_pretrained('/content/drive/MyDrive/colab_folder/models/pubmedbert/SWT_pubmedbert_model_15-11-2023')
tokenizer.save_pretrained('/content/drive/MyDrive/colab_folder/models/pubmedbert/SWT_pubmedbert_tokenizer_15-11-2023')

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1 B-cond,F1 I-cond,F1 B-des,F1 I-des,F1 B-subj,F1 I-subj,F1 B-group A,F1 I-group A,F1 B-group B,F1 I-group B,F1 B-group C,F1 I-group C,F1 B-group D,F1 I-group D
1,0.416100,0.205744,0.000000,0.491474,0.000000,0.617839,0.269231,0.087302,0.110329,0.000000,0.397338,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.200000,0.172218,0.496203,0.578089,0.000000,0.611174,0.452381,0.573460,0.448691,0.338462,0.503059,0.231214,nan,0.000000,0.000000,0.000000
3,0.160000,0.173613,0.539499,0.625000,0.506438,0.709512,0.496528,0.614565,0.530729,0.420063,0.545052,0.351351,0.122271,nan,0.000000,0.000000
4,0.128900,0.164039,0.583144,0.632530,0.576000,0.690722,0.527439,0.584112,0.474684,0.432122,0.547315,0.352705,0.228188,0.226415,0.000000,0.000000
5,0.103500,0.184374,0.569767,0.628420,0.648084,0.710018,0.510924,0.594378,0.521056,0.380256,0.566845,0.426667,0.153846,0.134454,0.000000,0.000000
6,0.086500,0.200722,0.585366,0.640203,0.608150,0.671756,0.588556,0.646825,0.519313,0.388592,0.562874,0.440678,0.160920,0.296296,0.000000,0.000000
7,0.073700,0.209159,0.571942,0.641026,0.605442,0.692460,0.597544,0.646617,0.532525,0.383659,0.559859,0.392324,0.133779,0.317460,0.000000,0.000000
8,0.066500,0.213443,0.588889,0.628217,0.608414,0.686380,0.585507,0.656064,0.534521,0.387097,0.558219,0.444860,0.136201,0.373832,0.000000,0.000000


<ipython-input-15-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)
<ipython-input-15-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)


('/content/drive/MyDrive/colab_folder/models/pubmedbert/SWT_pubmedbert_tokenizer_15-11-2023/tokenizer_config.json',
 '/content/drive/MyDrive/colab_folder/models/pubmedbert/SWT_pubmedbert_tokenizer_15-11-2023/special_tokens_map.json',
 '/content/drive/MyDrive/colab_folder/models/pubmedbert/SWT_pubmedbert_tokenizer_15-11-2023/vocab.txt',
 '/content/drive/MyDrive/colab_folder/models/pubmedbert/SWT_pubmedbert_tokenizer_15-11-2023/added_tokens.json',
 '/content/drive/MyDrive/colab_folder/models/pubmedbert/SWT_pubmedbert_tokenizer_15-11-2023/tokenizer.json')

#### Testing the model

In [37]:
trainer.evaluate(eval_dataset= tokenized_dataset['test'] )

{'eval_loss': 0.16086433827877045,
 'eval_f1_B-cond': 0.5484536082474226,
 'eval_f1_I-cond': 0.543757292882147,
 'eval_f1_B-des': 0.5917602996254683,
 'eval_f1_I-des': 0.6546146527117032,
 'eval_f1_B-subj': 0.5681818181818181,
 'eval_f1_I-subj': 0.5277044854881267,
 'eval_f1_B-group_A': 0.5949579831932774,
 'eval_f1_I-group_A': 0.35218093699515346,
 'eval_f1_B-group_B': 0.545822102425876,
 'eval_f1_I-group_B': 0.36942675159235666,
 'eval_f1_B-group_C': 0.2535211267605634,
 'eval_f1_I-group_C': 0.25301204819277107,
 'eval_f1_B-group_D': 0.0,
 'eval_f1_I-group_D': 0.0,
 'eval_runtime': 2.1911,
 'eval_samples_per_second': 70.74,
 'eval_steps_per_second': 4.564,
 'epoch': 8.0}

In [38]:
sentence = "Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown"
sentence

'Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown'

In [39]:
model_checkpoint = trainer.state.best_model_checkpoint # or save model in disk and load it later
print(f"using checkpoint {model_checkpoint}")

from transformers import pipeline

token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="first")
# token_classifier.tokenizer.model_max_length = model_max_length

using checkpoint /content/drive/MyDrive/colab_folder/models/checkpoint-312


In [40]:
res = token_classifier(sentence, aggregation_strategy="first")
print(res)

[{'entity_group': 'group_A', 'score': 0.35176525, 'word': 'semaglutide', 'start': 0, 'end': 11}, {'entity_group': 'subj', 'score': 0.7439712, 'word': 'patients', 'start': 127, 'end': 135}]


###  ABSTRACT AND METHODS

In [41]:
abstract_methods[0]

{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

##### count the annotations

In [42]:
for token, label in zip(abstract_methods[13]['tokens'], abstract_methods[13]['ner_tags']):
    print(f"{token:_<40} {label}")

alpha-dihydroergocryptine_______________ 7
in______________________________________ 0
the_____________________________________ 0
treatment_______________________________ 0
of______________________________________ 0
de______________________________________ 0
novo____________________________________ 0
parkinsonian____________________________ 0
patients________________________________ 5
:_______________________________________ 0
results_________________________________ 0
of______________________________________ 0
a_______________________________________ 0
multicentre_____________________________ 0
,_______________________________________ 0
randomized______________________________ 0
,_______________________________________ 0
double-blind____________________________ 0
,_______________________________________ 0
placebo-controlled______________________ 0
study___________________________________ 0
._______________________________________ 0
introduction____________________________ 0
:__________

In [43]:
tags = explore_annotations(abstract_methods)
import pandas as pd
result = count_values(tags)
temp_df = pd.DataFrame(result.items(), columns=['label_id', 'count'])
temp_df.sort_values(by = ['count'], ascending=False)

There are 2158415 annotations in the dataset


,label_id,count
0,O,2102999
1,B-group_A,7941
2,B-subjects,7582
10,I-condition,7058
7,B-group_B,6515
5,I-design,6379
3,I-subjects,4317
6,I-group_A,3799
9,B-condition,3119
11,I-group_B,2982


##### Saving the data as a json

In [44]:
len(abstract_methods)

1549

#### Creating Huggingface Dataset

In [45]:
abstract_methods_hf_ds = create_datasets(abstract_methods, tru_label)

Casting the dataset:   0%|          | 0/1549 [00:00<?, ? examples/s]

In [46]:
abstract_methods_hf_ds.features

{'pmid': Value(dtype='string', id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-cond', 'I-cond', 'B-des', 'I-des', 'B-subj', 'I-subj', 'B-group_A', 'I-group_A', 'B-group_B', 'I-group_B', 'B-group_C', 'I-group_C', 'B-group_D', 'I-group_D'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [47]:
abstract_methods_hf_ds[0]


{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [48]:
_abstract_methods_part_train_test = train_test_split(abstract_methods_hf_ds, train_test_size=0.2, validation_size=0.5)


In [49]:
_abstract_methods_part_train_test

DatasetDict({
    train: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 1239
    })
    test: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 155
    })
    valid: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 155
    })
})

In [50]:
tags = _abstract_methods_part_train_test['train'].features['ner_tags'].feature.names
tags

['O',
 'B-cond',
 'I-cond',
 'B-des',
 'I-des',
 'B-subj',
 'I-subj',
 'B-group_A',
 'I-group_A',
 'B-group_B',
 'I-group_B',
 'B-group_C',
 'I-group_C',
 'B-group_D',
 'I-group_D']

#### tokenizing dataset

In [51]:
tokenized_dataset = _abstract_methods_part_train_test.map(tokenize_align_labels, batched = True, remove_columns=_abstract_methods_part_train_test['train'].column_names)

Map:   0%|          | 0/1239 [00:00<?, ? examples/s]

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

#### Trainer

In [52]:
output_dir = r'/content/drive/MyDrive/colab_folder/models/pubmedbert/Abstract_Method'
run_name = "pubmedbert_Abstract_Method"
from transformers import Trainer
args = TrainingArguments(
    output_dir= output_dir,
    evaluation_strategy='epoch',
    eval_steps=1,
    save_strategy = 'epoch',
    learning_rate = learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    gradient_accumulation_steps=gradient_accumulation_steps,
    logging_strategy="epoch",
    save_total_limit=2,
    run_name =run_name,
    load_best_model_at_end=True
)


trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['valid'],
    data_collator = data_collator,
    tokenizer = tokenizer,


    compute_metrics = compute_metrics_by_token_swt

)

In [53]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1239
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 155
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 155
    })
})

In [54]:
trainer.train()

model.save_pretrained('/content/drive/MyDrive/colab_folder/models/pubmedbert/Abstract_method_SWT_pubmedbert_model_18-11-2023')
tokenizer.save_pretrained('/content/drive/MyDrive/colab_folder/models/pubmedbert/tokenizer/Abstract_method_SWT_pubmedbert_tokenizer_18-11-2023')

Epoch,Training Loss,Validation Loss,F1 B-cond,F1 I-cond,F1 B-des,F1 I-des,F1 B-subj,F1 I-subj,F1 B-group A,F1 I-group A,F1 B-group B,F1 I-group B,F1 B-group C,F1 I-group C,F1 B-group D,F1 I-group D
1,0.132900,0.122555,0.628337,0.669633,0.626016,0.759814,0.655378,0.539936,0.650735,0.552268,0.517034,0.571901,0.520833,0.457143,nan,0.000000
2,0.102300,0.133552,0.572298,0.664723,0.654244,0.785000,0.648588,0.527378,0.688060,0.527331,0.573302,0.539964,0.671835,0.431655,0.000000,0.000000
3,0.081000,0.139014,0.631579,0.704000,0.663830,0.768856,0.644831,0.547977,0.673634,0.510448,0.589206,0.503067,0.648045,0.428571,nan,0.000000
4,0.065200,0.141135,0.644007,0.684169,0.669623,0.785805,0.662921,0.519108,0.693347,0.508800,0.592777,0.491743,0.663158,0.418605,nan,0.000000
5,0.053200,0.145975,0.635688,0.685624,0.650943,0.741184,0.678201,0.560876,0.705329,0.501650,0.558322,0.457256,0.501718,0.475248,nan,nan
6,0.044600,0.162578,0.632143,0.677091,0.668113,0.758870,0.682540,0.550318,0.696663,0.500803,0.606700,0.560935,0.672269,0.370968,nan,nan
7,0.039200,0.169132,0.645161,0.682279,0.652874,0.743280,0.687361,0.557089,0.688525,0.487342,0.596705,0.549918,0.666667,0.344262,nan,nan
8,0.035300,0.172960,0.652482,0.685937,0.665188,0.741268,0.682222,0.551905,0.690909,0.492308,0.598176,0.541528,0.670391,0.402878,nan,nan


<ipython-input-15-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)
<ipython-input-15-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)
<ipython-input-15-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)
<ipython-input-15-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)
<ipython-input-15-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)
<ipython-input-15-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)
<ipython-input-15-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)


('/content/drive/MyDrive/colab_folder/models/pubmedbert/tokenizer/Abstract_method_SWT_pubmedbert_tokenizer_18-11-2023/tokenizer_config.json',
 '/content/drive/MyDrive/colab_folder/models/pubmedbert/tokenizer/Abstract_method_SWT_pubmedbert_tokenizer_18-11-2023/special_tokens_map.json',
 '/content/drive/MyDrive/colab_folder/models/pubmedbert/tokenizer/Abstract_method_SWT_pubmedbert_tokenizer_18-11-2023/vocab.txt',
 '/content/drive/MyDrive/colab_folder/models/pubmedbert/tokenizer/Abstract_method_SWT_pubmedbert_tokenizer_18-11-2023/added_tokens.json',
 '/content/drive/MyDrive/colab_folder/models/pubmedbert/tokenizer/Abstract_method_SWT_pubmedbert_tokenizer_18-11-2023/tokenizer.json')

#### Testing the model

In [55]:
trainer.evaluate(eval_dataset= tokenized_dataset['test'])

{'eval_loss': 0.12415549904108047,
 'eval_f1_B-cond': 0.6657101865136298,
 'eval_f1_I-cond': 0.5763052208835342,
 'eval_f1_B-des': 0.585820895522388,
 'eval_f1_I-des': 0.7485530546623794,
 'eval_f1_B-subj': 0.5987903225806451,
 'eval_f1_I-subj': 0.6124661246612466,
 'eval_f1_B-group_A': 0.7265795206971677,
 'eval_f1_I-group_A': 0.757192174913694,
 'eval_f1_B-group_B': 0.7290400972053462,
 'eval_f1_I-group_B': 0.5347432024169185,
 'eval_f1_B-group_C': 0.6746268656716418,
 'eval_f1_I-group_C': 0.26262626262626265,
 'eval_f1_B-group_D': 0.0,
 'eval_f1_I-group_D': 0.0,
 'eval_runtime': 2.3137,
 'eval_samples_per_second': 66.992,
 'eval_steps_per_second': 4.322,
 'epoch': 8.0}

In [56]:
sentence = "Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown"
sentence

'Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown'

In [57]:
model_checkpoint = trainer.state.best_model_checkpoint # or save model in disk and load it later
print(f"using checkpoint {model_checkpoint}")

from transformers import pipeline

token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="first")
# token_classifier.tokenizer.model_max_length = model_max_length

using checkpoint /content/drive/MyDrive/colab_folder/models/pubmedbert/Abstract_Method/checkpoint-78


In [58]:
res = token_classifier(sentence, aggregation_strategy="first")
print(res)

[{'entity_group': 'subj', 'score': 0.84700674, 'word': 'patients', 'start': 127, 'end': 135}]


###  ABSTRACT AND METHODS SPLITTED INTO CHUNKS OF 50

#### Creating a dataset for training

In [59]:
splitted_50_records_method_context[0]

{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [60]:
for token, label in zip(splitted_50_records_method_context[0]['tokens'], splitted_50_records_method_context[0]['ner_tags']):
    print(f"{token:_<40} {label}")

effects_________________________________ 0
of______________________________________ 0
rivastigmine____________________________ 7
in______________________________________ 0
patients________________________________ 0
with____________________________________ 0
and_____________________________________ 0
without_________________________________ 0
visual__________________________________ 0
hallucinations__________________________ 0
in______________________________________ 0
dementia________________________________ 0
associated______________________________ 0
with____________________________________ 0
parkinson’s_____________________________ 0
disease_________________________________ 0
._______________________________________ 0
we______________________________________ 0
aimed___________________________________ 0
to______________________________________ 0
determine_______________________________ 0
prospectively___________________________ 0
whether_________________________________ 0
rivastigmin

##### counting the annotation

In [61]:
tags = explore_annotations(splitted_50_records_method_context)

result = count_values(tags)
temp_df = pd.DataFrame(result.items(), columns=['label_id', 'count'])
temp_df.sort_values(by = ['count'], ascending=False)

There are 2158415 annotations in the dataset


,label_id,count
0,O,2103043
1,B-group_A,7938
2,B-subjects,7580
10,I-condition,7048
7,B-group_B,6514
5,I-design,6369
3,I-subjects,4309
6,I-group_A,3796
9,B-condition,3117
11,I-group_B,2979


In [62]:
len(splitted_50_records_method_context)

4170

#### Creating Huggingface Dataset

In [63]:
splitted_50_records_method_context_hf_ds = create_datasets(splitted_50_records_method_context, tru_label)

Casting the dataset:   0%|          | 0/4170 [00:00<?, ? examples/s]

In [64]:
splitted_50_records_method_context_hf_ds.features

{'pmid': Value(dtype='string', id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-cond', 'I-cond', 'B-des', 'I-des', 'B-subj', 'I-subj', 'B-group_A', 'I-group_A', 'B-group_B', 'I-group_B', 'B-group_C', 'I-group_C', 'B-group_D', 'I-group_D'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [65]:
splitted_50_records_method_context_hf_ds[0]


{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  9,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [66]:
splitted_50_records_method_context_train_test_valid = train_test_split(splitted_50_records_method_context_hf_ds, train_test_size=0.2, validation_size=0.5)


In [67]:
splitted_50_records_method_context_train_test_valid

DatasetDict({
    train: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 3336
    })
    test: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 417
    })
    valid: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 417
    })
})

In [68]:
tags = splitted_50_records_method_context_train_test_valid['train'].features['ner_tags'].feature.names
tags

['O',
 'B-cond',
 'I-cond',
 'B-des',
 'I-des',
 'B-subj',
 'I-subj',
 'B-group_A',
 'I-group_A',
 'B-group_B',
 'I-group_B',
 'B-group_C',
 'I-group_C',
 'B-group_D',
 'I-group_D']

#### tokenizing dataset

In [69]:
tokenized_dataset = splitted_50_records_method_context_train_test_valid.map(tokenize_align_labels, batched = True, remove_columns=splitted_50_records_method_context_train_test_valid['train'].column_names)

Map:   0%|          | 0/3336 [00:00<?, ? examples/s]

Map:   0%|          | 0/417 [00:00<?, ? examples/s]

Map:   0%|          | 0/417 [00:00<?, ? examples/s]

#### Trainer

In [70]:
output_dir = r'/content/drive/MyDrive/colab_folder/models/pubmedbert'
run_name = 'pubmedbert_50_method_abstract'
from transformers import Trainer
args = TrainingArguments(
    output_dir= output_dir,
    evaluation_strategy='epoch',
    eval_steps=1,
    save_strategy = 'epoch',
    learning_rate = learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    gradient_accumulation_steps=gradient_accumulation_steps,
    logging_strategy="epoch",
    save_total_limit=2,
    run_name =run_name,
    load_best_model_at_end=True
)


trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['valid'],
    data_collator = data_collator,
    tokenizer = tokenizer,


    compute_metrics = compute_metrics_by_token_swt)


In [71]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3336
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 417
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 417
    })
})

In [72]:
trainer.train()

model.save_pretrained('/content/drive/MyDrive/colab_folder/models/pubmedbert/Abstract_method_50_SWT_pubmedbert_model_15-11-2023')
tokenizer.save_pretrained('/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_50_SWT_pubmedbert_tokenizer_15-11-2023')

Epoch,Training Loss,Validation Loss,F1 B-cond,F1 I-cond,F1 B-des,F1 I-des,F1 B-subj,F1 I-subj,F1 B-group A,F1 I-group A,F1 B-group B,F1 I-group B,F1 B-group C,F1 I-group C,F1 B-group D,F1 I-group D
1,0.071700,0.064496,0.594393,0.630390,0.517949,0.710756,0.482629,0.493939,0.652810,0.501687,0.553360,0.521277,0.454128,0.556863,0.000000,0.000000
2,0.055500,0.063764,0.611722,0.656250,0.522523,0.702703,0.556908,0.518710,0.634779,0.599201,0.561528,0.475177,0.600000,0.443396,0.477876,0.060606
3,0.042800,0.065536,0.607634,0.664013,0.559809,0.688907,0.521038,0.529332,0.664706,0.620237,0.607306,0.541667,0.498054,0.397590,0.473684,nan
4,0.034100,0.070508,0.600610,0.637776,0.602020,0.708618,0.579259,0.536451,0.652239,0.633952,0.618243,0.576071,0.618076,0.543590,0.650307,0.622951
5,0.026600,0.078543,0.584759,0.635559,0.604082,0.743042,0.546862,0.529201,0.670455,0.623472,0.644258,0.572959,0.710692,0.548077,0.685315,0.000000
6,0.021500,0.079861,0.590504,0.628800,0.602851,0.725916,0.574614,0.529891,0.681556,0.632972,0.641657,0.549451,0.660606,0.596154,0.680272,0.653061
7,0.017800,0.079632,0.585600,0.653749,0.596421,0.730618,0.578431,0.536054,0.680620,0.630098,0.613731,0.564103,0.666667,0.570048,0.750000,0.716981
8,0.015400,0.083295,0.589666,0.653367,0.593220,0.733569,0.586371,0.540897,0.689291,0.615183,0.625767,0.571006,0.677852,0.576744,0.697987,0.716981


<ipython-input-15-5a6dcc639685>:11: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * prec * rec / (prec + rec)


('/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_50_SWT_pubmedbert_tokenizer_15-11-2023/tokenizer_config.json',
 '/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_50_SWT_pubmedbert_tokenizer_15-11-2023/special_tokens_map.json',
 '/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_50_SWT_pubmedbert_tokenizer_15-11-2023/vocab.txt',
 '/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_50_SWT_pubmedbert_tokenizer_15-11-2023/added_tokens.json',
 '/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_50_SWT_pubmedbert_tokenizer_15-11-2023/tokenizer.json')

#### Testing the model

In [73]:
trainer.evaluate(eval_dataset= tokenized_dataset['test'])

{'eval_loss': 0.06886200606822968,
 'eval_f1_B-cond': 0.6433770014556041,
 'eval_f1_I-cond': 0.6871727748691099,
 'eval_f1_B-des': 0.6072106261859582,
 'eval_f1_I-des': 0.7788461538461539,
 'eval_f1_B-subj': 0.6132361870066788,
 'eval_f1_I-subj': 0.6633291614518148,
 'eval_f1_B-group_A': 0.7031900138696255,
 'eval_f1_I-group_A': 0.7133574007220216,
 'eval_f1_B-group_B': 0.6094781043791242,
 'eval_f1_I-group_B': 0.5978755690440061,
 'eval_f1_B-group_C': 0.5792349726775957,
 'eval_f1_I-group_C': 0.5,
 'eval_f1_B-group_D': 0.4027777777777778,
 'eval_f1_I-group_D': 0.4415584415584415,
 'eval_runtime': 6.0899,
 'eval_samples_per_second': 68.474,
 'eval_steps_per_second': 4.434,
 'epoch': 8.0}

In [74]:
sentence = "Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown"
sentence

'Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown'

In [75]:
model_checkpoint = trainer.state.best_model_checkpoint # or save model in disk and load it later
print(f"using checkpoint {model_checkpoint}")

from transformers import pipeline

token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="first")
# token_classifier.tokenizer.model_max_length = model_max_length

using checkpoint /content/drive/MyDrive/colab_folder/models/pubmedbert/checkpoint-418


In [76]:
res = token_classifier(sentence, aggregation_strategy="first")
print(res)

[{'entity_group': 'subj', 'score': 0.70270646, 'word': 'patients', 'start': 127, 'end': 135}]


###  ABSTRACT AND METHODS SPLITTED INTO CHUNKS OF 500

#### Let create a data set with just condition, subject, design

In [77]:
splitted_500_abstract_methods_context[0]

{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['effects',
  'of',
  'rivastigmine',
  'in',
  'patients',
  'with',
  'and',
  'without',
  'visual',
  'hallucinations',
  'in',
  'dementia',
  'associated',
  'with',
  'parkinson’s',
  'disease',
  '.',
  'we',
  'aimed',
  'to',
  'determine',
  'prospectively',
  'whether',
  'rivastigmine',
  ',',
  'an',
  'inhibitor',
  'of',
  'acetylcholinesterase',
  'and',
  'butyrylcholinesterase',
  ',',
  'provided',
  'benefits',
  'in',
  'patients',
  'with',
  'and',
  'without',
  'visual',
  'hallucinations',
  'in',
  'a',
  'population',
  'with',
  'dementia',
  'associated',
  'with',
  "parkinson's",
  'disease',
  '(',
  'pdd',
  ')',
  '.',
  'this',
  

In [78]:
for token, label in zip(splitted_500_abstract_methods_context[0]['tokens'], splitted_500_abstract_methods_context[0]['ner_tags']):
    print(f"{token:_<40} {label}")

effects_________________________________ 0
of______________________________________ 0
rivastigmine____________________________ 7
in______________________________________ 0
patients________________________________ 0
with____________________________________ 0
and_____________________________________ 0
without_________________________________ 0
visual__________________________________ 0
hallucinations__________________________ 0
in______________________________________ 0
dementia________________________________ 0
associated______________________________ 0
with____________________________________ 0
parkinson’s_____________________________ 0
disease_________________________________ 0
._______________________________________ 0
we______________________________________ 0
aimed___________________________________ 0
to______________________________________ 0
determine_______________________________ 0
prospectively___________________________ 0
whether_________________________________ 0
rivastigmin

In [79]:

tags = explore_annotations(splitted_500_abstract_methods_context)

result = count_values(tags)
temp_df = pd.DataFrame(result.items(), columns=['label_id', 'count'])
temp_df.sort_values(by = ['count'], ascending=False)

There are 2158415 annotations in the dataset


,label_id,count
0,O,2107343
1,B-group_A,7834
2,B-subjects,7421
5,B-group_B,6403
9,I-condition,5841
10,I-design,5321
3,I-subjects,3767
4,I-group_A,3543
8,B-condition,2909
11,I-group_B,2700


##### Saving the data as a json

In [80]:
len(splitted_500_abstract_methods_context)

35678

#### Creating Huggingface Dataset

In [81]:
splitted_500_abstract_methods_context_hf_ds = create_datasets(splitted_500_abstract_methods_context, tru_label)

Casting the dataset:   0%|          | 0/35678 [00:00<?, ? examples/s]

In [82]:
splitted_500_abstract_methods_context_hf_ds.features

{'pmid': Value(dtype='string', id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-cond', 'I-cond', 'B-des', 'I-des', 'B-subj', 'I-subj', 'B-group_A', 'I-group_A', 'B-group_B', 'I-group_B', 'B-group_C', 'I-group_C', 'B-group_D', 'I-group_D'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [83]:
splitted_500_abstract_methods_context_hf_ds[0]


{'pmid': '16960863',
 'ner_tags': [0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  6,
  6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['effects',
  'of',
  'rivastigmine',
  'in',
  'patients',
  'with',
  'and',
  'without',
  'visual',
  'hallucinations',
  'in',
  'dementia',
  'associated',
  'with',
  'parkinson’s',
  'disease',
  '.',
  'we',
  'aimed',
  'to',
  'determine',
  'prospectively',
  'whether',
  'rivastigmine',
  ',',
  'an',
  'inhibitor',
  'of',
  'acetylcholinesterase',
  'and',
  'butyrylcholinesterase',
  ',',
  'provided',
  'benefits',
  'in',
  'patients',
  'with',
  'and',
  'without',
  'visual',
  'hallucinations',
  'in',
  'a',
  'population',
  'with',
  'dementia',
  'associated',
  'with',
  "parkinson's",
  'disease',
  '(',
  'pdd',
  ')',
  '.',
  'this',
  

In [84]:
splitted_500_abstract_methods_context_train_test_valid = train_test_split(splitted_500_abstract_methods_context_hf_ds, train_test_size=0.2, validation_size=0.5)


In [85]:
splitted_500_abstract_methods_context_train_test_valid

DatasetDict({
    train: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 28542
    })
    test: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 3568
    })
    valid: Dataset({
        features: ['pmid', 'ner_tags', 'tokens'],
        num_rows: 3568
    })
})

In [86]:
tags = splitted_500_abstract_methods_context_train_test_valid['train'].features['ner_tags'].feature.names
tags

['O',
 'B-cond',
 'I-cond',
 'B-des',
 'I-des',
 'B-subj',
 'I-subj',
 'B-group_A',
 'I-group_A',
 'B-group_B',
 'I-group_B',
 'B-group_C',
 'I-group_C',
 'B-group_D',
 'I-group_D']

#### tokenizing dataset

In [87]:
tokenized_dataset = splitted_500_abstract_methods_context_train_test_valid.map(tokenize_align_labels, batched = True, remove_columns=splitted_500_abstract_methods_context_train_test_valid['train'].column_names)

Map:   0%|          | 0/28542 [00:00<?, ? examples/s]

Map:   0%|          | 0/3568 [00:00<?, ? examples/s]

Map:   0%|          | 0/3568 [00:00<?, ? examples/s]

#### Trainer

In [88]:
output_dir = r'/content/drive/MyDrive/colab_folder/models/pubmedbert'
run_name = 'pubmedbert_500_method_abstract'
from transformers import Trainer
args = TrainingArguments(
    output_dir= output_dir,
    evaluation_strategy='epoch',
    eval_steps=1,
    save_strategy = 'epoch',
    learning_rate = learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    gradient_accumulation_steps=gradient_accumulation_steps,
    logging_strategy="epoch",
    save_total_limit=2,
    run_name =run_name,
    load_best_model_at_end=True
)


trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['valid'],
    data_collator = data_collator,
    tokenizer = tokenizer,


    compute_metrics = compute_metrics_by_token_swt)


In [89]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 28542
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3568
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3568
    })
})

In [90]:
trainer.train()

model.save_pretrained('/content/drive/MyDrive/colab_folder/models/pubmedbert/Abstract_method_500_SWT_pubmedbert_model_15-11-2023')
tokenizer.save_pretrained('/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_500_SWT_pubmedbert_tokenizer_15-11-2023')

Epoch,Training Loss,Validation Loss,F1 B-cond,F1 I-cond,F1 B-des,F1 I-des,F1 B-subj,F1 I-subj,F1 B-group A,F1 I-group A,F1 B-group B,F1 I-group B,F1 B-group C,F1 I-group C,F1 B-group D,F1 I-group D
1,0.075900,0.066912,0.501629,0.468832,0.615079,0.713842,0.355976,0.348148,0.562072,0.497992,0.621913,0.412281,0.536709,0.488189,0.101695,0.035714
2,0.056600,0.064705,0.575130,0.531080,0.583756,0.737675,0.441691,0.504382,0.688271,0.546108,0.650737,0.459843,0.532075,0.421053,0.703911,0.103448
3,0.041400,0.069191,0.584856,0.523772,0.633663,0.712799,0.404682,0.418367,0.691698,0.540984,0.661919,0.425273,0.600806,0.467532,0.708738,0.375000
4,0.029600,0.072635,0.571776,0.575107,0.606061,0.741481,0.514357,0.495356,0.693850,0.580367,0.688391,0.476190,0.635628,0.481481,0.733624,0.516129
5,0.021000,0.080375,0.584094,0.548105,0.593525,0.725788,0.491354,0.479763,0.715481,0.551043,0.679531,0.494815,0.669355,0.605578,0.834081,0.617886


Epoch,Training Loss,Validation Loss,F1 B-cond,F1 I-cond,F1 B-des,F1 I-des,F1 B-subj,F1 I-subj,F1 B-group A,F1 I-group A,F1 B-group B,F1 I-group B,F1 B-group C,F1 I-group C,F1 B-group D,F1 I-group D
1,0.075900,0.066912,0.501629,0.468832,0.615079,0.713842,0.355976,0.348148,0.562072,0.497992,0.621913,0.412281,0.536709,0.488189,0.101695,0.035714
2,0.056600,0.064705,0.575130,0.531080,0.583756,0.737675,0.441691,0.504382,0.688271,0.546108,0.650737,0.459843,0.532075,0.421053,0.703911,0.103448
3,0.041400,0.069191,0.584856,0.523772,0.633663,0.712799,0.404682,0.418367,0.691698,0.540984,0.661919,0.425273,0.600806,0.467532,0.708738,0.375000
4,0.029600,0.072635,0.571776,0.575107,0.606061,0.741481,0.514357,0.495356,0.693850,0.580367,0.688391,0.476190,0.635628,0.481481,0.733624,0.516129
5,0.021000,0.080375,0.584094,0.548105,0.593525,0.725788,0.491354,0.479763,0.715481,0.551043,0.679531,0.494815,0.669355,0.605578,0.834081,0.617886
6,0.014800,0.086670,0.591656,0.591304,0.627767,0.743383,0.553165,0.502674,0.744641,0.572997,0.685285,0.495050,0.718686,0.602941,0.807175,0.710744
7,0.010500,0.093180,0.581967,0.582339,0.623158,0.720437,0.548867,0.487594,0.726506,0.590062,0.693738,0.525316,0.665227,0.636735,0.774468,0.710744
8,0.007700,0.097956,0.593289,0.584468,0.630303,0.728242,0.548429,0.492881,0.733208,0.566857,0.703390,0.506536,0.684874,0.661417,0.794760,0.704918


('/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_500_SWT_pubmedbert_tokenizer_15-11-2023/tokenizer_config.json',
 '/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_500_SWT_pubmedbert_tokenizer_15-11-2023/special_tokens_map.json',
 '/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_500_SWT_pubmedbert_tokenizer_15-11-2023/vocab.txt',
 '/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_500_SWT_pubmedbert_tokenizer_15-11-2023/added_tokens.json',
 '/content/drive/MyDrive/colab_folder/tokenizer/pubmedbert/Abstract_method_500_SWT_pubmedbert_tokenizer_15-11-2023/tokenizer.json')

#### Testing the model

In [91]:
trainer.evaluate(eval_dataset= tokenized_dataset['test'])

{'eval_loss': 0.06692694872617722,
 'eval_f1_B-cond': 0.5095541401273884,
 'eval_f1_I-cond': 0.4701348747591522,
 'eval_f1_B-des': 0.4945652173913044,
 'eval_f1_I-des': 0.6662428480610298,
 'eval_f1_B-subj': 0.4832306639288158,
 'eval_f1_I-subj': 0.5410628019323671,
 'eval_f1_B-group_A': 0.6964193046185781,
 'eval_f1_I-group_A': 0.5425790754257908,
 'eval_f1_B-group_B': 0.6444263628966639,
 'eval_f1_I-group_B': 0.5565217391304348,
 'eval_f1_B-group_C': 0.6039453717754172,
 'eval_f1_I-group_C': 0.5533596837944664,
 'eval_f1_B-group_D': 0.588235294117647,
 'eval_f1_I-group_D': 0.2857142857142857,
 'eval_runtime': 9.3544,
 'eval_samples_per_second': 381.426,
 'eval_steps_per_second': 23.839,
 'epoch': 8.0}

In [92]:
sentence = "Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown"
sentence

'Semaglutide, a glucagon-like peptide-1 receptor agonist, has been shown to reduce the risk of adverse cardiovascular events in patients with diabetes. Whether semaglutide can reduce cardiovascular risk associated with overweight and obesity in the absence of diabetes is unknown'

In [93]:
model_checkpoint = trainer.state.best_model_checkpoint # or save model in disk and load it later
print(f"using checkpoint {model_checkpoint}")

from transformers import pipeline

token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="first")
# token_classifier.tokenizer.model_max_length = model_max_length

using checkpoint /content/drive/MyDrive/colab_folder/models/pubmedbert/checkpoint-3568


In [94]:
res = token_classifier(sentence, aggregation_strategy="first")
print(res)

[]
